In [1]:
import numpy as np

In [2]:
class DataNode():
    def __init__(self, activity_patterns):
        self.n_neurons=activity_patterns.shape[0]
        self.activity_patterns=activity_patterns
        
class SimuNode():
    def __init__(self,n_neurons):
        self.n_neurons=n_neurons
        
class Synapse():
    def __init__(self,pre_synaptic_node, post_synaptic_node,syn_type='exc',plasticity=None):
        self.pre_synaptic_node=pre_synaptic_node
        self.post_synaptic_node=post_synaptic_node
        self.syn_type=syn_type
        if plasticity=='BCM':
            self.theta_BCM = 0
            self.theta_BCM_dt = .001
            self.BCM_target = 5000.0
            self.alpha = 0.01
        
    def connect(self):
        row_dim=self.pre_synaptic_node.n_neurons
        col_dim=self.post_synaptic_node.n_neurons
        k=7.5
        theta=1.0
        if self.syn_type=='inh':
            self.syn_weights=-np.random.gamma(shape=k,scale=theta,size=(col_dim,row_dim))
            #self.syn_weights=-np.random.normal(loc=0,size=(col_dim,row_dim))
        elif self.syn_type=='exc':
            self.syn_weights=np.random.gamma(shape=k,scale=theta,size=(col_dim,row_dim))
            
    def learn(self,pre_syn_activities,post_syn_activities):
        if plasticity=='BCM':
            self._update_weights_BCM(pre_syn_activities,post_syn_activities)
            self._update_BCM_threshold(post_syn_activities)
        
    def _update_weights_BCM(self,pre_syn_activities,post_syn_activities):
        print('W before',self.syn_weights.shape)
        #print('syn',pre_syn_activity_pattern.reshape(9479,1)@post_syn_activity_pattern.reshape(1,2))
        self.syn_weights+= self.alpha*pre_syn_activities@post_syn_activities@(post_syn_activity_pattern-self.theta_BCM)
        print('W after',self.syn_weights.shape)
        
    def _update_BCM_threshold(self,activity_pattern):
        self.theta_BCM += self.theta_BCM_dt*((activity_pattern/self.BCM_target)*activity_pattern - self.theta_BCM)
        
class Simulate():
    def __init__(self,init_dict,data):
        self.init_dict=init_dict
        self.connectivity_graph=self.init_dict['connectivity_graph']
        self.node_classification=self.init_dict['node_classification']
        self.node_ns=self.init_dict['node_ns']
        self.data=data
        self.data_dict={}
        inh_dat_ind=0
        exc_dat_ind=0
        for j in range(len(self.node_classification)):
            if node=='data_inh':
                self.data_inds[j]=dat_ind
                inh_dat_ind+=1
            if node=='data_exc':
                self.data_inds[j]=exc_ind
                exc_dat_ind+=1
        
    def _connect(self):
        for node_ind,node in enumerate(self.node_classification):
            if node=='simulated_inh':
                node_lst.append(SimuNode(self.node_ns[node_ind]))
            if node=='simulated_exc':
                node_lst.append(SimuNode(self.node_ns[node_ind]))
            if node=='data_inh':
                ind=data_inds[node_ind]
                node_lst.append(DataNode(data[self.data[ind]))
            if node=='data_exc':
                ind=data_inds[node_ind]
                node_lst.append(data[self.data[ind])
        synapse_lst=[]
        for pre in range(0,self.connectivity_graph.shape[0]):
            for post in range(0,self.connectivity_graph.shape[1]):
                if self.connectivity_graph[pre,post]==1:
                    if node_classification[pre][-3:]=='exc':
                        syn_type='exc'
                    elif node_classification[pre][-3:]=='inh':
                        syn_type='inh'
                    print(syn_type)
                    syn=Synapse(node_lst[pre],node_lst[post],syn_type)
                    syn.connect()
                    synapse_lst.append(syn)
        
    def run(self):
        
        
        

In [ ]:
node_classification=['data_exc','data_inh','simulated_exc']
connectivity_graph=np.array([[0,0,1],[0,0,1],[0,0,1]])
node_ns=[100,10,2]

init_dict={'node_classification':node_classification,'connectivity_graph':connectivity_graph,'node_ns':node_ns}
run=Simulate(init_dict,data)